In [1]:
import numpy as np
from operator import itemgetter
import pandas as pd
import pathlib  
import html 
from bs4 import BeautifulSoup


In [4]:
def add_quotes(val):
    if isinstance(val, str) and not ('\"' in val):
        return f'\"{val}\"'
    return val


def stripHtmlTags(htmlTxt):
    if htmlTxt is None:
            return None
    else:
            return ''.join(BeautifulSoup(htmlTxt).findAll(text=True)) 



In [5]:
raw_paths = [  pathlib.Path().cwd()/"data" /"soc-pokec-profiles.txt", 
               pathlib.Path().cwd()/"data"/"soc-pokec-relationships.txt"]

node_fields = [
            "public",
            "completion_percentage",
            "gender",
            "region",
            "last_login",
            "registration",
            "AGE",
            "body",
            "I_am_working_in_field",
            "spoken_languages",
            "hobbies",
            "I_most_enjoy_good_food",
            "pets",
            "body_type",
            "my_eyesight",
            "eye_color",
            "hair_color",
            "hair_type",
            "completed_level_of_education",
            "favourite_color",
            "relation_to_smoking",
            "relation_to_alcohol",
            "sign_in_zodiac",
            "on_pokec_i_am_looking_for",
            "love_is_for_me",
            "relation_to_casual_sex",
            "my_partner_should_be",
            "marital_status",
            "children",
            "relation_to_children",
            "I_like_movies",
            "I_like_watching_movie",
            "I_like_music",
            "I_mostly_like_listening_to_music",
            "the_idea_of_good_evening",
            "I_like_specialties_from_kitchen",
            "fun",
            "I_am_going_to_concerts",
            "my_active_sports",
            "my_passive_sports",
            "profession",
            "I_like_books",
            "life_style",
            "music",
            "cars",
            "politics",
            "relationships",
            "art_culture",
            "hobbies_interests",
            "science_technologies",
            "computers_internet",
            "education",
            "sport",
            "movies",
            "travelling",
            "health",
            "companies_brands",
            "more",
            ""
        ]
dfn = pd.read_csv(raw_paths[0], sep = "\t", names = node_fields, nrows = None)
dfe = pd.read_csv(raw_paths[1], sep = "\t", names = ["source", "target"], nrows = None)


# dfn["last_login"] = dfn["last_login"].apply(lambda row: f"""dateTime({row})""" )
# dfn["registration"] = dfn["registration"].apply(lambda row: f"""dateTime({row})""") 
# transform edges
dfe = dfe.astype({'source': 'str', 'target': 'str'})


# save as hdf
# store["dfn"] = dfn
# store["dfe"] = dfe
# store.close()
len(dfn) * len(node_fields)

96335377

In [6]:
node_frame = dfn[["public", "completion_percentage", "gender", "region","AGE","last_login","registration"]]
edge_frame = dfe

In [7]:
node_frame.update(node_frame[['region', 'last_login',"registration"]].map('"{}"'.format))

In [8]:
edge_lable = ["Follow", "Favorite","Anonymously"]
node_frame

,public,completion_percentage,gender,region,AGE,last_login,registration
1,1,14,1.0,"""zilinsky kraj, zilina""",26.0,"""2012-05-25 11:20:00.0""","""2005-04-03 00:00:00.0"""
2,1,62,0.0,"""zilinsky kraj, kysucke nove mesto""",0.0,"""2012-05-25 23:08:00.0""","""2007-11-30 00:00:00.0"""
16,1,64,1.0,"""zilinsky kraj, kysucke nove mesto""",23.0,"""2012-05-25 23:19:40.0""","""2008-05-18 00:00:00.0"""
3,0,38,1.0,"""bratislavsky kraj, bratislava - karlova ves""",29.0,"""2012-05-10 18:05:00.0""","""2010-05-23 00:00:00.0"""
4,1,12,0.0,"""banskobystricky kraj, brezno""",26.0,"""2011-12-29 12:25:00.0""","""2011-12-29 00:00:00.0"""
...,...,...,...,...,...,...,...
1632799,1,12,0.0,"""banskobystricky kraj, revuca""",23.0,"""2012-05-12 16:46:00.0""","""2012-05-12 00:00:00.0"""
1632800,1,12,1.0,"""trenciansky kraj, myjava""",33.0,"""2012-02-09 02:43:00.0""","""2012-02-06 00:00:00.0"""
1632801,1,12,1.0,"""kosicky kraj, kosice - okolie""",0.0,"""2012-05-26 12:10:00.0""","""2009-07-19 00:00:00.0"""
1632802,1,16,1.0,"""bratislavsky kraj, bratislava - karlova ves""",19.0,"""2012-05-16 11:32:00.0""","""2011-05-01 00:00:00.0"""


In [9]:
# transform nodes
import random
node_strings = node_frame.apply(
    lambda row: f"N{row.name}:people "+' '.join([f"{col}:{row[col]}" for col in node_frame.columns if pd.notna(row[col])]),
    axis=1
)

edge_strings = dfe.apply(
    lambda row: f"N{row["source"]} -> N{row["target"]}: {random.choice(edge_lable)}",
    axis=1
)


In [10]:
with open(pathlib.Path().cwd()/"data" /"soc-pokec2.qm",'w') as f:
    for row in node_strings:
        f.write(row + "\n")
    for row in edge_strings:
        f.write(row + "\n")

   

In [11]:
print(node_strings[6])

N6:people public:0 completion_percentage:22 gender:0.0 region:"zilinsky kraj, zilina" AGE:38.0 last_login:"2012-05-25 20:40:00.0" registration:"2007-12-15 00:00:00.0"
